## Go (Even More) Hands On!

__You already have learned enough to do some SparkML modeling__

So let's dive into a real data set...

####UCI ML Repository Combined Cycle Power Plant Data Set 

Adapted from: https://archive.ics.uci.edu/ml/datasets/combined+cycle+power+plant

Features consist of hourly average ambient variables 
* Temperature (T) in the range 1.81°C and 37.11°C,
* Ambient Pressure (AP) in the range 992.89-1033.30 milibar,
* Relative Humidity (RH) in the range 25.56% to 100.16%
* Exhaust Vacuum (V) in teh range 25.36-81.56 cm Hg
* Net hourly electrical energy output (PE) 420.26-495.76 MW

The averages are taken from various sensors located around the plant that record the ambient variables every second. 

The variables are given without normalization.

*Let's try and predict (model) the power output (PE) as a function of the other 4 variables.*

To make it easier to get started, especially since DBFS and the Spark DataFrame readers may be new to you, we'll walk through finding and ingesting the data:

In [4]:
%sh ls /dbfs/databricks-datasets/power-plant/data/

Sheet1.tsv
Sheet2.tsv
Sheet3.tsv
Sheet4.tsv
Sheet5.tsv

In [5]:
%sh head /dbfs/databricks-datasets/power-plant/data/Sheet1.tsv

AT	V	AP	RH	PE
14.96	41.76	1024.07	73.17	463.26
25.18	62.96	1020.04	59.08	444.37
5.11	39.4	1012.16	92.14	488.56
20.86	57.32	1010.24	76.64	446.48
10.82	37.5	1009.23	96.62	473.9
26.27	59.44	1012.23	58.77	443.67
15.89	43.96	1014.02	75.24	467.35
9.48	44.71	1019.12	66.43	478.42
14.64	45	1021.78	41.25	475.98

In [6]:
spark.read.text("dbfs:/databricks-datasets/power-plant/data/").show(truncate=False)

+--------------------------------+
value |
+--------------------------------+
AT	V	AP	RH	PE |
14.96	41.76	1024.07	73.17	463.26|
25.18	62.96	1020.04	59.08	444.37|
5.11	39.4	1012.16	92.14	488.56 |
20.86	57.32	1010.24	76.64	446.48|
10.82	37.5	1009.23	96.62	473.9 |
26.27	59.44	1012.23	58.77	443.67|
15.89	43.96	1014.02	75.24	467.35|
9.48	44.71	1019.12	66.43	478.42 |
14.64	45	1021.78	41.25	475.98 |
11.74	43.56	1015.14	70.72	477.5 |
17.99	43.72	1008.64	75.04	453.02|
20.14	46.93	1014.66	64.22	453.99|
24.34	73.5	1011.31	84.15	440.29 |
25.71	58.59	1012.77	61.83	451.28|
26.19	69.34	1009.48	87.59	433.99|
21.42	43.79	1015.76	43.08	462.19|
18.21	45	1022.86	48.84	467.54 |
11.04	41.74	1022.6	77.51	477.2 |
14.45	52.75	1023.97	63.59	459.85|
+--------------------------------+
only showing top 20 rows

In [7]:
data = spark.read.option("delimiter", "\t") \
          .option("header", True) \
          .option("inferSchema", True) \
          .csv("dbfs:/databricks-datasets/power-plant/data/")

data.show()

+-----+-----+-------+-----+------+
 AT| V| AP| RH| PE|
+-----+-----+-------+-----+------+
14.96|41.76|1024.07|73.17|463.26|
25.18|62.96|1020.04|59.08|444.37|
 5.11| 39.4|1012.16|92.14|488.56|
20.86|57.32|1010.24|76.64|446.48|
10.82| 37.5|1009.23|96.62| 473.9|
26.27|59.44|1012.23|58.77|443.67|
15.89|43.96|1014.02|75.24|467.35|
 9.48|44.71|1019.12|66.43|478.42|
14.64| 45.0|1021.78|41.25|475.98|
11.74|43.56|1015.14|70.72| 477.5|
17.99|43.72|1008.64|75.04|453.02|
20.14|46.93|1014.66|64.22|453.99|
24.34| 73.5|1011.31|84.15|440.29|
25.71|58.59|1012.77|61.83|451.28|
26.19|69.34|1009.48|87.59|433.99|
21.42|43.79|1015.76|43.08|462.19|
18.21| 45.0|1022.86|48.84|467.54|
11.04|41.74| 1022.6|77.51| 477.2|
14.45|52.75|1023.97|63.59|459.85|
13.97|38.47|1015.15|55.28| 464.3|
+-----+-----+-------+-----+------+
only showing top 20 rows

In [8]:
from pyspark.ml.feature import *

assembler = VectorAssembler(inputCols=["AT","V","AP","RH"], outputCol="features")

assembler.transform(data).show()

+-----+-----+-------+-----+------+--------------------+
 AT| V| AP| RH| PE| features|
+-----+-----+-------+-----+------+--------------------+
14.96|41.76|1024.07|73.17|463.26|[14.96,41.76,1024...|
25.18|62.96|1020.04|59.08|444.37|[25.18,62.96,1020...|
 5.11| 39.4|1012.16|92.14|488.56|[5.11,39.4,1012.1...|
20.86|57.32|1010.24|76.64|446.48|[20.86,57.32,1010...|
10.82| 37.5|1009.23|96.62| 473.9|[10.82,37.5,1009....|
26.27|59.44|1012.23|58.77|443.67|[26.27,59.44,1012...|
15.89|43.96|1014.02|75.24|467.35|[15.89,43.96,1014...|
 9.48|44.71|1019.12|66.43|478.42|[9.48,44.71,1019....|
14.64| 45.0|1021.78|41.25|475.98|[14.64,45.0,1021....|
11.74|43.56|1015.14|70.72| 477.5|[11.74,43.56,1015...|
17.99|43.72|1008.64|75.04|453.02|[17.99,43.72,1008...|
20.14|46.93|1014.66|64.22|453.99|[20.14,46.93,1014...|
24.34| 73.5|1011.31|84.15|440.29|[24.34,73.5,1011....|
25.71|58.59|1012.77|61.83|451.28|[25.71,58.59,1012...|
26.19|69.34|1009.48|87.59|433.99|[26.19,69.34,1009...|
21.42|43.79|1015.76|43.08|462.19|[21.42,43.79,1015...|
18.21| 45.0|1022.86|48.84|467.54|[18.21,45.0,1022....|
11.04|41.74| 1022.6|77.51| 477.2|[11.04,41.74,1022...|
14.45|52.75|1023.97|63.59|459.85|[14.45,52.75,1023...|
13.97|38.47|1015.15|55.28| 464.3|[13.97,38.47,1015...|
+-----+-----+-------+-----+------+--------------------+
only showing top 20 rows

In [9]:
from pyspark.ml.regression import *

lr = LinearRegression(labelCol="PE")

In [10]:
train, test = data.randomSplit([0.75, 0.25])

lrModel = lr.fit ( assembler.transform(train) )

lrModel.transform( assembler.transform(test) ).show()

+----+-----+-------+-----+------+--------------------+------------------+
 AT| V| AP| RH| PE| features| prediction|
+----+-----+-------+-----+------+--------------------+------------------+
 3.2|41.31| 997.67|98.84|489.86|[3.2,41.31,997.67...| 484.9429637236955|
3.21|38.44| 1016.9|86.34|491.35|[3.21,38.44,1016....| 488.7683514979698|
3.31|39.42|1024.05|84.31|487.19|[3.31,39.42,1024....| 489.1117146876702|
 3.6|35.19|1018.73| 99.1|488.98|[3.6,35.19,1018.7...| 486.8506361383351|
3.73|39.42| 1024.4|82.42|488.58|[3.73,39.42,1024....|488.60104020140005|
3.82|35.47|1016.62|84.34|489.04|[3.82,35.47,1016....| 488.5488823120404|
3.91|35.47|1016.92|86.03|488.67|[3.91,35.47,1016....|488.12272687121765|
3.96|35.47|1016.79|83.81|489.68|[3.96,35.47,1016....| 488.3662087938835|
3.99|39.64|1011.53|83.58|492.06|[3.99,39.64,1011....| 487.0457349772013|
3.99| 39.9|1009.74|96.81|490.91|[3.99,39.9,1009.7...|484.78402785073735|
4.08|35.19|1018.87|97.07|489.44|[4.08,35.19,1018....| 486.2301845705362|
4.15| 39.9|1007.62|95.69| 489.8|[4.15,39.9,1007.6...| 484.5114271553109|
4.32|35.47| 1017.8|88.51|488.03|[4.32,35.47,1017....| 486.9749788410391|
4.41|42.07|1005.54|68.85|488.36|[4.41,42.07,1005....|487.60135643168513|
4.49|38.44|1015.47|76.96|485.78|[4.49,38.44,1015....| 487.6271798803928|
4.71|39.42| 1026.4|84.42|481.28|[4.71,39.42,1026....|486.47141050425705|
4.71|42.07|1005.35|82.26|486.79|[4.71,42.07,1005....| 484.8782249332327|
4.73|41.31| 999.77|93.44| 486.6|[4.73,41.31,999.7...| 482.8998546930416|
4.87|42.85| 1013.6| 92.3|482.22|[4.87,42.85,1013....|483.31221738373415|
4.95|42.07|1004.87|80.88|485.67|[4.95,42.07,1004....| 484.5911670820105|
+----+-----+-------+-----+------+--------------------+------------------+
only showing top 20 rows

In [11]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[assembler, lr])


In [12]:
model = pipeline.fit(train)

In [13]:
summary = model.stages[-1].summary
print(summary.r2)
print(summary.rootMeanSquaredError)

0.9298630034054072
4.516183017746002

In [14]:
from pyspark.ml.evaluation import *

eval = RegressionEvaluator(labelCol="PE", predictionCol="prediction")

In [15]:
predictions = model.transform(test)
eval.evaluate(predictions)

Out[ 19 ]: 4.678222536642573

In [16]:
data.describe().show()

+-------+------------------+------------------+------------------+------------------+------------------+
summary| AT| V| AP| RH| PE|
+-------+------------------+------------------+------------------+------------------+------------------+
 count| 47840| 47840| 47840| 47840| 47840|
 mean|19.651231187290996| 54.30580372073594|1013.2590781772572| 73.30897784280918|454.36500940635506|
 stddev| 7.452161658340004|12.707361709685806| 5.938535418520816|14.599658352081477| 17.06628146683769|
 min| 1.81| 25.36| 992.89| 25.56| 420.26|
 max| 37.11| 81.56| 1033.3| 100.16| 495.76|
+-------+------------------+------------------+------------------+------------------+------------------+

In [17]:
# basic steps are...

# split off a test set

# assemble predictor columns into a vector column

# fit a LinearRegression to the training set data frame

# see how well it worked using an Evaluator and the test data